In [ ]:
# Mais informações confira No Medium
# https://medium.com/@conversandoemcodigos/limpando-os-dados-dos-gastos-do-senado-7-days-of-code-dia-01-6477112b5c8d

In [ ]:
#importei o panda para poder usar

import pandas as pd

In [ ]:
# subi os arquivos
# o encoding serve para deixar normalizar os acentos e cedilhas

# os dados podem ser baixados no site:  
# https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps

despesa_2019 = pd.read_csv('despesa_ceaps_2019.csv', encoding = 'ANSI')
despesa_2020 = pd.read_csv('despesa_ceaps_2020.csv', encoding = 'ANSI')
despesa_2021 = pd.read_csv('despesa_ceaps_2021.csv', encoding = 'ANSI')
despesa_2022 = pd.read_csv('despesa_ceaps_2022.csv', encoding = 'ANSI')

In [ ]:
#concatenei os arquivos. Usei o ignore_index como True para que juntasse tudo em um index só

despesa_total = pd.concat([despesa_2019, despesa_2020, despesa_2021, despesa_2022], ignore_index=True)

In [ ]:
#fazendo uma cópia pra não mexer no dataset original

despesa_limpo = despesa_total

In [ ]:
#apagando as colunas que não vou usar

despesa_limpo = despesa_limpo.drop(columns=['DOCUMENTO'])
despesa_limpo = despesa_limpo.drop(columns=['COD_DOCUMENTO', 'CNPJ_CPF'])

In [ ]:
# convertendo os arquivos para o formato certo
# errors = coerce, faz com que os valores que não estejam nesse formato fiquem nulos

despesa_limpo['DATA'] = pd.to_datetime(despesa_limpo['DATA'], errors = 'coerce')

In [ ]:
#inplace -> para gravar mudanças no dataset
#regex -> para poder substituir apenas uma parte da mensagem

despesa_limpo['VALOR_REEMBOLSADO'].replace(',', '.', regex=True, inplace=True)

In [ ]:
#convertendo para float

despesa_limpo['VALOR_REEMBOLSADO'] = despesa_limpo['VALOR_REEMBOLSADO'].astype(float)

In [ ]:
despesa_limpo.sample(10)

In [ ]:
# Convertendo ano e mês em string, porque não serão feito contas com eles
despesa_limpo['ANO'] = despesa_limpo['ANO'].astype(str)
despesa_limpo['MES'] = despesa_limpo['MES'].astype(str)

In [ ]:
#Verificando se todas os dados estão no formato correto
despesa_limpo.info()

In [ ]:
# Vendo as datas não nulas

mascara_data_nulo = despesa_limpo['DATA'].isnull()
despesa_limpo[mascara_data_nulo]

In [ ]:
#preenchendo os valores nulos em detalhamento
#inplace -> faz mudança no arquivo

despesa_limpo['DETALHAMENTO'].fillna('Nao Preenchido', inplace=True)

In [ ]:
import plotly.express as px

In [ ]:
# Verificando se há outliers

fig = px.box(despesa_limpo, y="VALOR_REEMBOLSADO")
fig.show()

In [ ]:
# Organizando os dados por valores e verificando os outliers

despesa_limpo.sort_values(by='VALOR_REEMBOLSADO')

In [ ]:
# filtrando dados

despesa_limpo_sem_outliers = despesa_limpo.query('10 < VALOR_REEMBOLSADO < 2000000')

In [ ]:

fig = px.box(despesa_limpo_sem_outliers, y="VALOR_REEMBOLSADO")
fig.show()

In [ ]:
# Verificando quais informações não teve o detalhamento preenchido

detalhamento_nao_preenchido = despesa_limpo_sem_outliers.query('DETALHAMENTO == "Nao Preenchido"')

In [ ]:
fig = px.box(detalhamento_nao_preenchido, y="VALOR_REEMBOLSADO")
fig.show()

In [ ]:
#exportação do arquivo

despesa_limpo_sem_outliers.to_csv('despesa_limpo.csv', index=False)
detalhamento_nao_preenchido.to_csv('despesa_limpo_np.csv', index=False)